In [1]:
import brightway2 as bw
import wurst as w
import pandas as pd
import csv

Match STEM labels to ecoinvent activities. We don't need to load the whole database into memory for this simple example.

In [2]:
db = bw.Database("ecoinvent 3.3 cutoff")

Load STEM-ecoinvent matching lists.

In [3]:
df = pd.read_excel("stem-to-ecoinvent.xlsx")
df

,STEM name,Ecoinvent 3.3,Location
0,Gas (B),"electricity production, natural gas, combined ...",DE
1,Gas (F),"electricity production, natural gas, combined ...",DE
2,Gas (CHP),"heat and power co-generation, biogas, gas engine",CH
3,Gas-CCS,"electricity production, natural gas, combined ...",DE
4,Coal,"electricity production, hard coal",DE
5,Coal-CCS,"electricity production, hard coal",DE
6,Nuclear,"electricity production, nuclear, pressure wate...",CH
7,Oil,"electricity production, oil",DE
8,Hydro (R),"electricity production, hydro, run-of-river",CH
9,Hydro (D),"electricity production, hydro, reservoir, alp...",CH


Get labels in the same order.

In [4]:
labels = [x[0].strip() for x in df.values]

Find the ecoinvent activities, using name, location, and unit.

In [5]:
mapped = {
    label.strip(): w.get_one(db, 
                             w.equals('name', name.strip()), 
                             w.equals('location', location.strip()),
                             w.equals('unit', 'kilowatt hour'))
    for label, name, location in df.values
}
mapped

{'Biomass': 'heat and power co-generation, wood chips, 6667 kW' (kilowatt hour, CH, None),
 'CHP (DG)': 'heat and power co-generation, wood chips, 6667 kW' (kilowatt hour, CH, None),
 'Coal': 'electricity production, hard coal' (kilowatt hour, DE, None),
 'Coal-CCS': 'electricity production, hard coal' (kilowatt hour, DE, None),
 'Gas (B)': 'electricity production, natural gas, combined cycle power plant' (kilowatt hour, DE, None),
 'Gas (CHP)': 'heat and power co-generation, biogas, gas engine' (kilowatt hour, CH, None),
 'Gas (F)': 'electricity production, natural gas, combined cycle power plant' (kilowatt hour, DE, None),
 'Gas-CCS': 'electricity production, natural gas, combined cycle power plant' (kilowatt hour, DE, None),
 'Geothermal': 'electricity production, deep geothermal' (kilowatt hour, CH, None),
 'Hydro (D)': 'electricity production, hydro, reservoir, alpine region' (kilowatt hour, CH, None),
 'Hydro (P)': 'electricity production, hydro, pumped storage' (kilowatt hour, C

Calculate scores for these methods (can be changed).

In [6]:
selected_methods = [
    ('IPCC 2013', 'climate change', 'GWP 100a'),
    ('ReCiPe Endpoint (H,A)', 'total', 'total'),
]

In [7]:
results = []

for method in selected_methods:
    for technology in mapped:
        lca = bw.LCA({mapped[technology]: 1}, method)
        lca.lci()
        lca.lcia()
        results.append(("-".join(method), technology, lca.score))

Export to CSV

In [8]:
with open('stem-scores.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    for row in results:
        writer.writerow(row)